# Workstream 1: Olist Data Engineering Pipeline (PoC)
**Mục tiêu:** Hợp nhất 9 tệp dữ liệu Olist thành một Bảng Dữ liệu Tổng thể (Master Table) duy nhất và trích xuất các đặc trưng nghiệp vụ (features) và Phân tích làm sạch dữ liệu với các quy trình tiêu chuẩn


In [19]:
# --- Bước 0: Khởi tạo và Tải Thư viện ---
import pandas as pd
import numpy as np
import os # Dùng để quản lý đường dẫn file

# Cấu hình Pandas để hiển thị tất cả các cột
pd.set_option('display.max_columns', None)
print("Các thư viện đã sẵn sàng.")

Các thư viện đã sẵn sàng.


In [20]:
# %% [code]
# --- Bước 1: Tải các tệp CSV chính ---

# Định nghĩa đường dẫn (Giả sử các tệp .csv nằm trong thư mục con 'data')
DATA_DIR = 'data/'

try:
    df_orders = pd.read_csv(os.path.join(DATA_DIR, 'olist_orders_dataset.csv'))
    df_items = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_items_dataset.csv'))
    df_products = pd.read_csv(os.path.join(DATA_DIR, 'olist_products_dataset.csv'))
    df_customers = pd.read_csv(os.path.join(DATA_DIR, 'olist_customers_dataset.csv'))
    df_reviews = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_reviews_dataset.csv'))
    df_payments = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_payments_dataset.csv'))
    df_sellers = pd.read_csv(os.path.join(DATA_DIR, 'olist_sellers_dataset.csv'))

    print("Tải 8 tệp dữ liệu chính thành công.")
    print(f"Orders shape: {df_orders.shape}")
    print(f"Items shape: {df_items.shape}")
    print(f"Payments shape: {df_payments.shape}")

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy thư mục 'data'. Vui lòng kiểm tra lại đường dẫn.")

Tải 8 tệp dữ liệu chính thành công.
Orders shape: (99441, 8)
Items shape: (112650, 7)
Payments shape: (103886, 5)


## Bước 2: (QUAN TRỌNG) Xử lý "Bẫy Hợp nhất" 💣 - Bảng `order_payments`

**Vấn đề:** Bảng `payments` có quan hệ 1-Nhiều (1 `order_id` có thể có nhiều hàng thanh toán, ví dụ: 1 voucher + 1 thẻ tín dụng). Nếu merge trực tiếp với `items` (cũng là 1-Nhiều), dữ liệu sẽ bị nhân lên sai lệch.

**Giải pháp:** Chúng ta phải **aggregate (gộp)** bảng `payments` theo `order_id` *trước khi* merge.

In [21]:
# %% [code]
print(f"Số hàng ban đầu của Payments: {df_payments.shape[0]}")
print(f"Số order_id duy nhất trong Payments: {df_payments['order_id'].nunique()}")

# Gộp (Aggregate) bảng payments
df_payments_agg = df_payments.groupby('order_id').agg(
    # Lấy tổng số lần trả góp
    payment_installments_total=('payment_installments', 'sum'),
    # Lấy tổng giá trị thanh toán
    payment_value_total=('payment_value', 'sum'),
    # Lấy loại thanh toán chính (ví dụ: 'credit_card', 'boleto')
    payment_type_primary=('payment_type', 'first')
).reset_index()

print(f"Số hàng sau khi gộp (agg): {df_payments_agg.shape[0]}")
print("-> Bảng Payments (df_payments_agg) đã an toàn để merge (1-1 với Orders).")

Số hàng ban đầu của Payments: 103886
Số order_id duy nhất trong Payments: 99440
Số hàng sau khi gộp (agg): 99440
-> Bảng Payments (df_payments_agg) đã an toàn để merge (1-1 với Orders).


## Bước 3: Thực thi Pipeline Hợp nhất Lõi (Core Merge Pipeline)

Chúng ta sẽ xây dựng Bảng Tổng thể (Master Table) theo Sơ đồ (Schema) đã vẽ.
Luồng (Flow): Bắt đầu từ `df_orders` và `merge` các bảng khác vào.

In [22]:
# %% [code]
print("Bắt đầu pipeline hợp nhất...")

# 1. Bảng gốc
df_master = df_orders.copy()
print(f"Shape ban đầu (Orders): {df_master.shape}")

# 2. Merge Customers (1-to-1)
df_master = pd.merge(df_master, df_customers, on='customer_id', how='left')
print(f"Shape sau khi merge Customers: {df_master.shape}")

# 3. Merge Reviews (1-to-1, giả định mỗi order 1 review chính)
# (Chúng ta chỉ lấy review cuối cùng nếu có nhiều review cho 1 order)
df_reviews_dedup = df_reviews.drop_duplicates(subset='order_id', keep='last')
df_master = pd.merge(df_master, df_reviews_dedup, on='order_id', how='left')
print(f"Shape sau khi merge Reviews: {df_master.shape}")

# 4. Merge Payments (đã xử lý) (1-to-1)
df_master = pd.merge(df_master, df_payments_agg, on='order_id', how='left')
print(f"Shape sau khi merge Payments Agg: {df_master.shape}")

# 5. (QUAN TRỌNG) Merge Items (1-to-Many)
# Đây là lúc bảng sẽ "nổ" (explode) ra, mỗi hàng là một item trong 1 order.
df_master = pd.merge(df_master, df_items, on='order_id', how='left')
print(f"Shape sau khi merge Items (Nổ 1-Nhiều): {df_master.shape}")

# 6. Merge Products (Many-to-1)
df_master = pd.merge(df_master, df_products, on='product_id', how='left')
print(f"Shape sau khi merge Products: {df_master.shape}")

# 7. Merge Sellers (Many-to-1)
df_master = pd.merge(df_master, df_sellers, on='seller_id', how='left')
print(f"Shape cuối cùng (Master Table): {df_master.shape}")

# 8. (CẮT TỈA) Bỏ qua Geolocation
# Ghi chú: Chúng ta chủ động bỏ qua bảng 'geolocation' (1 triệu hàng)
# trong PoC V1 này để tối ưu tốc độ. Chúng ta sẽ dùng 'customer_state'
# và 'seller_state' làm proxy (thay thế) cho vị trí địa lý.

print("\nKiểm tra 5 hàng đầu của Bảng Tổng thể:")
display(df_master.head())

Bắt đầu pipeline hợp nhất...
Shape ban đầu (Orders): (99441, 8)
Shape sau khi merge Customers: (99441, 12)
Shape sau khi merge Reviews: (99441, 18)
Shape sau khi merge Payments Agg: (99441, 21)
Shape sau khi merge Items (Nổ 1-Nhiều): (113425, 27)
Shape sau khi merge Products: (113425, 35)
Shape cuối cùng (Master Table): (113425, 38)

Kiểm tra 5 hàng đầu của Bảng Tổng thể:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_installments_total,payment_value_total,payment_type_primary,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,3.0,38.71,credit_card,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,1.0,141.46,boleto,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,3.0,179.12,credit_card,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,14840.0,guariba,SP
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,359d03e676b3c069f62cadba8dd3f6e8,5.0,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58,1.0,72.20,credit_card,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,31842.0,belo horizonte,MG
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,e50934924e227544ba8246aeb3770dd4,5.0,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51,1.0,28.62,credit_card,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0,8752.0,mogi das cruzes,SP


## Bước 4: Trích xuất Đặc trưng Nghiệp vụ (Feature Engineering)

Đây là bước "làm giàu" (enrichment) dữ liệu cho **Output Loại 1**.
Chúng ta tạo ra các đặc trưng (features) mà Workstream 2 (Dự báo) sẽ cần.

In [23]:
# %% [code]
print("Bắt đầu Feature Engineering...")

# 1. Xử lý Dữ liệu Thời gian (Timestamps)
# Chuyển đổi các cột quan trọng sang kiểu datetime
time_cols = ['order_purchase_timestamp', 'order_approved_at',
             'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in time_cols:
    df_master[col] = pd.to_datetime(df_master[col], errors='coerce')

# 2. Tạo Đặc trưng Vận hành (Logistics Features)
# (QUAN TRỌNG) Thời gian giao hàng thực tế (tính bằng ngày)
df_master['delivery_time_days'] = (df_master['order_delivered_customer_date'] - df_master['order_purchase_timestamp']).dt.total_seconds() / (24 * 60 * 60) # Tính chính xác bằng ngày (float)

# (QUAN TRỌNG) Giao hàng sớm hay trễ so với dự kiến
df_master['delivery_vs_estimated_days'] = (df_master['order_estimated_delivery_date'] - df_master['order_delivered_customer_date']).dt.total_seconds() / (24 * 60 * 60)
# (Feature này > 0 là giao sớm, < 0 là giao trễ)

# 3. Tạo Đặc trưng Vận chuyển (Freight Features)
# (QUAN TRỌNG) Tỷ lệ phí ship / giá sản phẩm
df_master['freight_ratio'] = df_master['freight_value'] / (df_master['price'] + 1e-6) # + 1e-6 để tránh chia cho 0

# 4. Tạo Đặc trưng Thanh toán (Payment Features)
# Tạo biến nhị phân (0/1) cho các loại thanh toán
df_master['is_payment_credit_card'] = (df_master['payment_type_primary'] == 'credit_card').astype(int)
df_master['is_payment_boleto'] = (df_master['payment_type_primary'] == 'boleto').astype(int)
df_master['is_payment_voucher'] = (df_master['payment_type_primary'] == 'voucher').astype(int)

# (QUAN TRỌNG) Khách hàng có trả góp không?
df_master['is_payment_installments'] = (df_master['payment_installments_total'].fillna(0) > 1).astype(int)

# 5. Tạo Đặc trưng Đánh giá (Review Features)
# (Cách làm PoC đơn giản, dùng .transform để gán review trung bình của sản phẩm/người bán)
# Cảnh báo: Cách này có rò rỉ dữ liệu (data leakage) một chút, nhưng chấp nhận được cho PoC V1.
df_master['avg_review_score_product'] = df_master.groupby('product_id')['review_score'].transform('mean')
df_master['avg_review_score_seller'] = df_master.groupby('seller_id')['review_score'].transform('mean')

print("Tạo 4 nhóm đặc trưng thành công (Logistics, Freight, Payment, Review).")
display(df_master[['price', 'freight_value', 'freight_ratio', 'delivery_time_days', 'delivery_vs_estimated_days', 'review_score', 'avg_review_score_product']].describe())

Bắt đầu Feature Engineering...
Tạo 4 nhóm đặc trưng thành công (Logistics, Freight, Payment, Review).


,price,freight_value,freight_ratio,delivery_time_days,delivery_vs_estimated_days,review_score,avg_review_score_product
count,112650.000000,112650.000000,112650.000000,110196.000000,110196.000000,112464.000000,112443.000000
mean,120.653739,19.990320,0.320864,12.472631,11.333149,4.017206,4.031617
std,183.633928,15.806405,0.349894,9.445655,10.162342,1.399829,0.872365
min,0.850000,0.000000,0.000000,0.533414,-188.975081,1.000000,1.000000
25%,39.900000,13.080000,0.134034,6.736262,6.476803,4.000000,3.750000
50%,74.990000,16.260000,0.231356,10.184282,12.046788,5.000000,4.142857
75%,134.900000,21.150000,0.393036,15.541097,16.321042,5.000000,4.617216
max,6735.000000,409.680000,26.235263,209.628611,146.016123,5.000000,5.000000


 Bước 5: Hoàn thiện PoC - Output Loại 1 (Sản phẩm Bàn giao)

Chúng ta sẽ làm sạch và lưu trữ Bảng Đặc trưng (Feature Table) cuối cùng.
Đây là sản phẩm của Workstream 1.

In [31]:
# %% [code]
# 1. Làm sạch: (Giữ nguyên code làm sạch)
print(f"Số hàng trước khi làm sạch: {df_master.shape[0]}")
df_master_clean = df_master.dropna(subset=['order_id', 'product_id', 'order_purchase_timestamp'])
print(f"Số hàng sau khi làm sạch: {df_master_clean.shape[0]}")

# 2. Định nghĩa cột cuối cùng: (Giữ nguyên code định nghĩa cột)
final_columns = [
    # IDs
    'order_id', 'product_id', 'customer_id', 'seller_id','order_item_id',

    # Time
    'order_purchase_timestamp',

    # Các đặc trưng (features) đã tạo
    'delivery_time_days', 'delivery_vs_estimated_days',
    'price', 'freight_value', 'freight_ratio',
    'is_payment_credit_card', 'is_payment_boleto', 'is_payment_installments', 'payment_value_total',
    'review_score', 'avg_review_score_product', 'avg_review_score_seller',

    # Các đặc trưng phân loại (Categorical features)
    'product_category_name', 'customer_state', 'seller_state'
]
final_columns_exist = [col for col in final_columns if col in df_master_clean.columns]
df_final_output = df_master_clean[final_columns_exist]

# 3. (THAY ĐỔI) Lưu trữ Output sang CSV
OUTPUT_FILE_CSV = 'olist_master_feature_table_v1.csv'
print(f"Đang lưu trữ file {OUTPUT_FILE_CSV} (việc này có thể mất vài phút)...")

# index=False là RẤT QUAN TRỌNG để không lưu chỉ số (index) của Pandas
df_final_output.to_csv(OUTPUT_FILE_CSV, index=False)

print(f"\n--- 🥳 HOÀN THÀNH WORKSTREAM 1 ---")
print(f"Output Loại 1 (Bảng Đặc trưng) đã được lưu tại: {OUTPUT_FILE_CSV}")
print(f"Kích thước Bảng Đặc trưng cuối cùng: {df_final_output.shape}")
display(df_final_output.head())

Số hàng trước khi làm sạch: 113425
Số hàng sau khi làm sạch: 112650
Đang lưu trữ file olist_master_feature_table_v1.csv (việc này có thể mất vài phút)...

--- 🥳 HOÀN THÀNH WORKSTREAM 1 ---
Output Loại 1 (Bảng Đặc trưng) đã được lưu tại: olist_master_feature_table_v1.csv
Kích thước Bảng Đặc trưng cuối cùng: (112650, 21)


,order_id,product_id,customer_id,seller_id,order_item_id,order_purchase_timestamp,delivery_time_days,delivery_vs_estimated_days,price,freight_value,freight_ratio,is_payment_credit_card,is_payment_boleto,is_payment_installments,payment_value_total,review_score,avg_review_score_product,avg_review_score_seller,product_category_name,customer_state,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,9ef432eb6251297304e76186b10a928d,3504c0cb71d7fa48d967e0e4c94d59d9,1.00,2017-10-02 10:56:33,8.44,7.11,29.99,8.72,0.29,1,0,1,38.71,4.00,4.00,4.42,utilidades_domesticas,SP,SP
1,53cdb2fc8bc7dce0b6741e2150273451,595fac2a385ac33a80bd5114aec74eb8,b0830fb4747a6c6d20dea0b8c802d7ef,289cdb325fb7e7f891c38608bf9e0962,1.00,2018-07-24 20:41:37,13.78,5.36,118.70,22.76,0.19,0,1,0,141.46,4.00,4.41,4.58,perfumaria,BA,SP
2,47770eb9100c2d0c44946d9cf07ec65d,aa4383b373c6aca5d8797843e5594415,41ce2a54c0b03bf3443c3d931a367089,4869f7a5dfa277a7dca6462dcf3b52b2,1.00,2018-08-08 08:38:49,9.39,17.25,159.90,19.22,0.12,1,0,1,179.12,5.00,5.00,4.12,automotivo,GO,SP
3,949d5b44dbf5de918fe9c16f97b45f8a,d0b61bfb1de832b15ba9d266ca96e5b0,f88197465ea7920adcdbec7375364d82,66922902710d126a0e7d26b0e3805106,1.00,2017-11-18 19:28:06,13.21,12.98,45.00,27.20,0.60,1,0,0,72.20,5.00,4.25,4.45,pet_shop,RN,MG
4,ad21c59c0840e6cb83a9ceb5573f8159,65266b2da20d04dbe00c5c2d3bb7859e,8ab97904e6daea8866dbdbc4fb7aad2c,2c9e548be18521d1c43cde1c582c6de8,1.00,2018-02-13 21:18:39,2.87,9.24,19.90,8.72,0.44,1,0,0,28.62,5.00,3.73,3.92,papelaria,SP,SP
